In [1]:
#!pip install beautifulsoup4
#!pip install requests
#!conda install pandas
#!pip install tqdm

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [25]:
def clean_text(text):
    return text.replace('/', '').strip().lower()


def process_mode(row):
    return clean_text(row.td.text)


def process_times(row):
    return tuple(clean_text(col.text) for col in row.find_all('td'))


def process_conj(row, mode, times, infinitive):
    verbs = []
    cols = row.find_all('td')
    
    for i in range(0, 6, 2):
        verbs.append(
            dict(infinitive=infinitive,
                 pess=cols[i].text,
                 conj=cols[i + 1].text,
                 time=times[i // 2],
                 mode=mode
                )
        )
        
    return verbs



def get_indicativo(infinitive:str, table:list) -> list:
    
    verbs = []
    
    for i, row in enumerate(table.find_all('tr')):
        if i == 3:
            mode = process_mode(row)
        if i == 4 or i == 12:
            times = process_times(row)
        if 5 <= i <= 10 or 14 <= i <= 19:
            verbs.extend(process_conj(row, mode, times, infinitive))
    
    return verbs

def get_conjugations(verb):
    try:
        r = requests.get(f"http://www.conjuga-me.net/verbo-{verb}")
        soup = BeautifulSoup(r.text, 'html.parser')
        indicativo = get_indicativo(verb, soup.find(class_="conj"))
        return indicativo
    except AttributeError:
        print(f"verb '{verb}' not found")
        return []

In [14]:
def clean_verb(verb):
    return verb.replace('[', '').replace(']', '').lower().strip()

#verbs = pd.read_clipboard().assign(verb=lambda df: df.verb.apply(clean_verb)).verb.values
verbs = pd.read_csv('top_1000_verbs_pt.csv').dropna().verbs.values
verbs[:1]

array(['ser'], dtype=object)

In [10]:
parsed_verbs = verbs[:1]

In [30]:
for i in range(500, 1001, 100):
    print(f"downloading {i}")

    parsed = []

    for verb in tqdm(verbs[i - 100:i]):
        parsed.extend(get_conjugations(verb))

    pd.DataFrame(parsed).to_csv(f'top_{i}_pt.csv', index=False)

downloading 500


verb 'efectuar' not found
downloading 600


downloading 700


verb 'seleccionar' not found
verb 'detectar' not found
downloading 800


downloading 900


verb 'enfatizar' not found
verb 'leccionar' not found
verb 'baptizar' not found
verb 'actualizar' not found
downloading 1000


verb 'accionar' not found


In [31]:
len(verbs)

999

In [29]:
pd.DataFrame(parsed).to_csv('top_400_pt.csv', index=False)

In [18]:
pd.DataFrame(verbs).rename(columns={0:'verbs'}).to_csv('top_1000_verbs_pt.csv', index=False)